<a href="https://colab.research.google.com/github/bkvkrll/Machine-Learning-in-Business/blob/main/Step1_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The Myers Briggs Type Indicator  https://www.kaggle.com/datasnaek/mbti-type?select=mbti_1.csv

In [1]:
pip install razdel

In [2]:
pip install pymorphy2

     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 8.2 MB 10.1 MB/s 


In [3]:
import pandas as pd

import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import itertools
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
import re
from nltk.corpus import stopwords
from razdel import tokenize
import pymorphy2
import nltk

In [4]:
comments = pd.read_csv("mbti_1.csv")
print(comments.shape)
comments['posts'][0]

(8675, 2)


"'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments  https://www.youtube.com/watch?v=iz7lE1g4XM4  sportscenter not top ten plays  https://www.youtube.com/watch?v=uCdfze1etec  pranks|||What has been the most life-changing experience in your life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8   http://www.youtube.com/watch?v=u8ejam5DP3E  On repeat for most of today.|||May the PerC Experience immerse you.|||The last thing my INFJ friend posted on his facebook before committing suicide the next day. Rest in peace~   http://vimeo.com/22842206|||Hello ENFJ7. Sorry to hear of your distress. It's only natural for a relationship to not be perfection all the time in every moment of existence. Try to figure the hard times as times of growth, as...|||84389  84390  http://wallpaperpassion.com/upload/23700/friendship-boy-and-girl-wallpaper.jpg  http://assets.dornob.com/wp-content/uploads/2010/04/round-home-design.jpg ...

In [5]:
comments.head(3)

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...


In [6]:
#text preprocessing
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
stopword_eng = stopwords.words('english')
print(len(stopword_eng))

morph = pymorphy2.MorphAnalyzer()

179


In [8]:
# stoplist file stop_word.txt
with open('stop_word.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_eng += additional_stopwords
len(stopword_eng)

2868

In [9]:
list_of_types = ['ENFJ', 'ENFP', 'ENTJ', 'ENTP', 'ESFJ', 'ESFP', 'ESTJ', 'ESTP', 'INFJ', 'INFP', 'INTJ', 'INTP', 'ISFJ', 'ISFP', 'ISTJ', 'ISTP']
for type in list_of_types:
    stopword_eng.append(type.lower())

In [10]:
len(stopword_eng)

2884

In [11]:
f_1 = open("final_stop_words.txt", 'a', encoding="utf-8")
f_1.write(stopword_eng[0])
for word in stopword_eng[1:]:
    f_1.write('\n')
    f_1.write(word)
f_1.close()

In [12]:
#text cleaning
def clean_text(text):

    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))
    text = re.sub("\|", ' ', str(text))
    text = re.sub("https?:\/\/\S+", '', str(text))
    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    

    return text

cache = {}

def lemmatization(text):


    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_eng] # [6]
    
    return words_lem_without_stopwords

In [13]:
text = clean_text(comments['posts'][0])
text2 = lemmatization(text)
print(text2)

['moments', 'sportscenter', 'plays', 'pranks', 'lifechanging', 'experience', 'life', 'repeat', 'today', 'perc', 'experience', 'immerse', 'friend', 'posted', 'facebook', 'committing', 'suicide', 'day', 'rest', 'peace', 'hear', 'distress', 'natural', 'relationship', 'perfection', 'time', 'moment', 'existence', 'figure', 'hard', 'times', 'times', 'growth', 'stuff', 'game', 'set', 'match', 'prozac', 'wellbrutin', 'minutes', 'moving', 'legs', 'moving', 'sitting', 'desk', 'chair', 'weed', 'moderation', 'edibles', 'healthier', 'alternative', 'basically', 'items', 'determined', 'type', 'types', 'types', 'cognitive', 'functions', 'whatnot', 'left', 'moderation', 'sims', 'video', 'game', 'good', 'note', 'good', 'subjective', 'completely', 'promoting', 'death', 'sim', 'dear', 'favorite', 'video', 'games', 'growing', 'current', 'favorite', 'video', 'games', 'cool', 'appears', 'late', 'sad', 'wait', 'thought', 'confidence', 'good', 'cherish', 'time', 'solitude', 'bc', 'revel', 'time', 'workin', 'en

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Possible nested set at position 39
  if sys.path[0] == '':


In [14]:
%%time
#text cleaning
comments['posts'] = comments['posts'].apply(lambda x: clean_text(x), 1)

CPU times: user 25 s, sys: 110 ms, total: 25.1 s
Wall time: 32.4 s


In [15]:
#lemmatization
comments['posts'] = comments['posts'].apply(lambda x: lemmatization(x), 1)

In [16]:
comments['posts']

0       [moments, sportscenter, plays, pranks, lifecha...
1       [finding, lack, posts, alarming, sex, boring, ...
2       [good, blessing, curse, absolutely, positive, ...
3       [dear, enjoyed, conversation, day, esoteric, g...
4       [fired, silly, misconception, approaching, log...
                              ...                        
8670    [ixfp, cats, fi, doms, reason, websites, neo, ...
8671    [soif, thread, exists, someplace, heck, delete...
8672    [questions, purple, pill, pick, winning, lotte...
8673    [conflicted, wanting, children, honestly, mate...
8674    [long, personalitycafe, changed, bit, good, tu...
Name: posts, Length: 8675, dtype: object

In [17]:
comments['posts'] = comments['posts'].apply(lambda x: ' '.join(x), 1)

In [18]:
comments['posts']

0       moments sportscenter plays pranks lifechanging...
1       finding lack posts alarming sex boring positio...
2       good blessing curse absolutely positive friend...
3       dear enjoyed conversation day esoteric gabbing...
4       fired silly misconception approaching logicall...
                              ...                        
8670    ixfp cats fi doms reason websites neo nazis pe...
8671    soif thread exists someplace heck delete ooops...
8672    questions purple pill pick winning lottery num...
8673    conflicted wanting children honestly maternal ...
8674    long personalitycafe changed bit good turn doc...
Name: posts, Length: 8675, dtype: object

In [19]:
comments['type'].value_counts() 

INFP    1832
INFJ    1470
INTP    1304
INTJ    1091
ENTP     685
ENFP     675
ISTP     337
ISFP     271
ENTJ     231
ISTJ     205
ENFJ     190
ISFJ     166
ESTP      89
ESFP      48
ESFJ      42
ESTJ      39
Name: type, dtype: int64

In [20]:
X_train, X_test, y_train, y_test = train_test_split(comments, comments['type'], random_state=0)

In [21]:
#Multiclass classification with GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [22]:
#make pipeline
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]

pipeline = Pipeline([('title_selector', FeatureSelector(column='posts')), 
                     ('title_tfidf', TfidfVectorizer()), 
                     ('clf', GradientBoostingClassifier())])

In [23]:
#learn pipline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('title_selector', FeatureSelector(column='posts')),
                ('title_tfidf', TfidfVectorizer()),
                ('clf', GradientBoostingClassifier())])

In [24]:
#prediction
preds = pipeline.predict(X_test)
preds[:10]

array(['INFP', 'INFJ', 'ENFP', 'INTJ', 'ENFP', 'INFP', 'INFP', 'ISFJ',
       'INTJ', 'INFJ'], dtype=object)

In [25]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score

In [26]:
models_results = {
    'approach': [],
    'f1_score_micro': [],
    'f1_score_macro': [],
    'f1_score_weighted': [],
    'precision_score_micro': [],
    'precision_score_macro': [],
    'precision_score_weighted': []
}

In [27]:
#result
models_results['approach'].append('Multi_Grad_Boost')
models_results['f1_score_micro'].append(f1_score(y_test, preds, average='micro'))
models_results['f1_score_macro'].append(f1_score(y_test, preds, average='macro'))
models_results['f1_score_weighted'].append(f1_score(y_test, preds, average='weighted'))
models_results['precision_score_micro'].append(precision_score(y_test, preds, average='micro'))
models_results['precision_score_macro'].append(precision_score(y_test, preds, average='macro'))
models_results['precision_score_weighted'].append(precision_score(y_test, preds, average='weighted'))

print(models_results)

{'approach': ['Multi_Grad_Boost'], 'f1_score_micro': [0.46519133241124944], 'f1_score_macro': [0.30729166105479166], 'f1_score_weighted': [0.45505870321938974], 'precision_score_micro': [0.46519133241124944], 'precision_score_macro': [0.3515579448793392], 'precision_score_weighted': [0.4658229998579148]}


In [37]:
#save model
import dill

with open("model_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

In [38]:
!pip freeze > "requirements.txt"